# Processamento e análise dos dados


In [1]:
# Preparando ambiente (importando bibliotecas e downloads...)

!pip install nltk
import nltk
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
import re
import pandas as pd 
import numpy as np
import spacy
nlp = spacy.load('pt')

[nltk_data] Downloading package rslp to /home/karine/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Importando dataset
df = pd.read_csv("dataset.csv", sep="," , encoding="utf8")
df = df[:10]

In [19]:
df

,text_pt,Competitivo,Ofensivo,Rebelde,Retraído,Subsequente,Cooperativo,Ajudante,Líder
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Este é um exemplo do motivo pelo qual a maiori...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Nem mesmo os Beatles puderam escrever músicas ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Filmes de fotos de latão não é uma palavra apr...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Uma coisa engraçada aconteceu comigo enquanto ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Este filme de terror alemão tem que ser um dos...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Sendo um fã de longa data do cinema japonês, e...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"""Tokyo Eyes"" fala de uma menina japonesa de 17...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Fazendeiros ricos em Buenos Aires têm uma long...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
degree_words = open('degree-words.txt','r', encoding='utf8')

### Removendo e tratando dados

In [6]:
# Remove columns e create column
df.drop(columns=['id', 'text_en', 'sentiment'], axis=1, inplace=True)

### Léxicos de Sentimentos

In [9]:
dicionario_lexico = {}

In [10]:
sentilexpt = open('SentiLex-lem-PT02.txt','r', encoding='utf8')

In [11]:
for i in sentilexpt:
    pos_ponto = i.find('.')
    palavra = (i[:pos_ponto])
    pol_pos = i.find('POL')
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')
    dicionario_lexico[palavra] = polaridade

In [12]:
ontolp = open('lexico_v3.0.txt','r', encoding='utf8')

In [13]:
for i in ontolp:
    split_dic = i.split(',')
    palavra = split_dic[0]
    if palavra not in dicionario_lexico:
        polaridade = split_dic[2]
        dicionario_lexico[palavra] = polaridade        

In [25]:
degree_words = open('degree-words.txt','r', encoding='utf8')
degree_words_set = {}

In [26]:
for i in degree_words:
    split_dic = i.split(';')
    degree_words_set[split_dic[0]] = split_dic[1][:-1]

### PLN

In [33]:
for frase in df['text_pt']:
    
    frase = nlp(frase)
    degree_words_head = []
    degree_words_aux = []
    score_frase = {'pos':0, 'neutro':0, 'neg':0}
    
    for token in frase:
        if(token.text in degree_words_set):
            degree_words_head.append(token.head.text)
            degree_words_aux.append(int(degree_words_set.get(token.text)))
            
    for token in frase:
        if(token.text in dicionario_lexico):
            if token.head.text in degree_words_head:
                dic = int(dicionario_lexico.get(token.text))
                neg = int(degree_words_aux[degree_words_head.index(token.head.text)])                   
                if dic == +1:
                    score_frase['pos'] = int(score_frase.get('pos')) + dic + neg
                elif dic == -1:
                    score_frase['neg'] = int(score_frase.get('neg')) + dic - neg
                else:
                    score_frase['neutro'] = int(score_frase.get('neutro')) + dic + neg
    print(score_frase)

{'pos': 4, 'neutro': 0, 'neg': 0}
{'pos': 4, 'neutro': 2, 'neg': -2}
{'pos': 0, 'neutro': 0, 'neg': -7}
{'pos': 3, 'neutro': 0, 'neg': 0}
{'pos': 10, 'neutro': 6, 'neg': 0}
{'pos': 0, 'neutro': 2, 'neg': -3}
{'pos': 6, 'neutro': 0, 'neg': -11}
{'pos': 20, 'neutro': 5, 'neg': -14}
{'pos': 0, 'neutro': 8, 'neg': 0}
{'pos': 4, 'neutro': 6, 'neg': -6}
